<a href="https://colab.research.google.com/github/datastax/ragstack-ai/blob/main/examples/notebooks/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quickstart with RAGStack

This notebook demonstrates how to set up a simple RAG pipeline with RAGStack. At the end of this notebook, you will have a fully functioning Question/Answer model that can answer questions using your supplied documents.

A RAG pipeline requires, at minimum, a vector store, an embedding model, and an LLM. In this tutorial, you will use an Astra DB vector store, an OpenAI embedding model, an OpenAI LLM, and LangChain to orchestrate it all together.

## Prerequisites

You will need a vector-enabled Astra database and an OpenAI Account.

* Create an [Astra vector database](https://docs.datastax.com/en/astra-serverless/docs/getting-started/create-db-choices.html).
* Create an [OpenAI account](https://openai.com/)
* Within your database, create an [Astra DB Access Token](https://docs.datastax.com/en/astra-serverless/docs/manage/org/manage-tokens.html) with Database Administrator permissions.
* Get your Astra DB Endpoint:
  * `https://<ASTRA_DB_ID>-<ASTRA_DB_REGION>.apps.astra.datastax.com`

See the [Prerequisites](https://docs.datastax.com/en/ragstack/docs/prerequisites.html) page for more details.

## Setup
`ragstack-ai` includes all the packages you need to build a RAG pipeline.

`datasets` is used to import a sample dataset

In [ ]:
! pip install -q ragstack-ai datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 29.3 

In [ ]:
import cassio

In [ ]:
import os
from getpass import getpass

# Enter your settings for Astra DB and OpenAI:
os.environ["ASTRA_DB_API_ENDPOINT"] ="https://5e8e8f77-9996-4395--1929ad6471c2--2.apps.astra.datastax.com"
os.environ["ASTRA_DB_APPLICATION_TOKEN"] ="AstraCS:MruNflHwfIwgINWbItH:695130d3a158b3832f44674762035af76db48dec3fed3c784c5df54742"
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")

Enter your OpenAI API Key: ··········


## Create RAG Pipeline

### Embedding Model and Vector Store

In [ ]:
from langchain_astradb import AstraDBVectorStore
from langchain.embeddings import OpenAIEmbeddings
import os

# Configure your embedding model and vector store
embedding = OpenAIEmbeddings()
vstore = AstraDBVectorStore(
    collection_name="test",
    embedding=embedding,
    token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
    api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
)
print("Astra vector store configured")

Astra vector store configured


In [ ]:
from datasets import load_dataset

# Load a sample dataset
philo_dataset = load_dataset("datastax/philosopher-quotes")["train"]
print("An example entry:")
print(philo_dataset[16])

Generating train split:   0%|          | 0/450 [00:00<?, ? examples/s]

An example entry:
{'author': 'aristotle', 'quote': 'Love well, be loved and do something of value.', 'tags': 'love;ethics'}


In [ ]:
from langchain.schema import Document

# Constructs a set of documents from your data. Documents can be used as inputs to your vector store.
docs = []
for entry in philo_dataset:
    metadata = {"author": entry["author"]}
    if entry["tags"]:
        # Add metadata tags to the metadata dictionary
        for tag in entry["tags"].split(";"):
            metadata[tag] = "y"
    # Create a LangChain document with the quote and metadata tags
    doc = Document(page_content=entry["quote"], metadata=metadata)
    docs.append(doc)

In [ ]:
docs

[Document(page_content="True happiness comes from gaining insight and growing into your best possible self. Otherwise all you're having is immediate gratification pleasure, which is fleeting and doesn't grow you as a person.", metadata={'author': 'aristotle', 'knowledge': 'y'}),
 Document(page_content='The roots of education are bitter, but the fruit is sweet.', metadata={'author': 'aristotle', 'education': 'y', 'knowledge': 'y'}),
 Document(page_content='Before you heal the body you must first heal the mind', metadata={'author': 'aristotle', 'ethics': 'y'}),
 Document(page_content='The proof that you know something is that you are able to teach it', metadata={'author': 'aristotle', 'education': 'y', 'knowledge': 'y'}),
 Document(page_content='Those who are not angry at the things they should be angry at are thought to be fools, and so are those who are not angry in the right way, at the right time, or with the right persons.', metadata={'author': 'aristotle'}),
 Document(page_content=

In [ ]:
# Create embeddings by inserting your documents into the vector store.
inserted_ids = vstore.add_documents(docs)
print(f"\nInserted {len(inserted_ids)} documents.")


Inserted 450 documents.


In [ ]:
# Checks your collection to verify the documents are embedded.
print(vstore.astra_db.collection("test").find())

{'data': {'documents': [{'_id': 'e4631988e2c247aaa01b6e0bc9d24a21', 'content': 'He who does not desire power is fit to hold it.', '$vector': [-0.00812149840956011, -0.026237692373499107, 0.00924282296292763, -0.006189969268813257, -0.03668609851074957, 0.01848564592585526, -0.011440099893788356, 0.0006514051150331121, 0.005036236550577017, -0.010195624686448749, 0.040782498745794996, 0.002560248982750108, 0.016333741170868842, -0.005720049946894178, 0.005538564037364816, 0.005308465880532374, 0.037204632873939616, -0.004401035867224251, 0.021207935500118312, -0.017435620639050603, 0.01257438752683884, -0.0040834352927172106, -0.005110775422907337, -0.008665957069470823, -0.018252306766271417, 0.0036459243684975095, 0.01901714057476038, -0.03168227338467047, 0.019574563245676687, -0.03844910714995659, 0.029115540766602967, -0.001895880671618705, -0.006306638848605177, -0.023243171606635436, -0.03310823263889418, 0.00015363520105296917, -0.013754046404441004, -0.002391726385734366, 0.024

### Basic Retrieval

Retrieve context from your vector database, and pass it to the model with a prompt.

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retriever = vstore.as_retriever(search_kwargs={"k": 3})

prompt_template = """
Answer the question based only on the supplied context. If you don't know the answer, say you don't know the answer.
Context: {context}
Question: {question}
Your answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)
model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("In the given context, what is the most important to allow the brain and provide me the tags?")

"The most important thing is to allow the brain the full measure of sleep required to restore it. The tags associated with the context are 'ethics' and 'knowledge'."

In [ ]:
# Add your questions here!
# chain.invoke("<your question>")

## Cleanup

In [ ]:
# WARNING: This will delete the collection and all documents in the collection
vstore.delete_collection()

You now have a fully functioning RAG pipeline! Note that there are several different ways to accomplish this, depending on your input data format, vector store, embedding, model, output type, and more. There are also more advanced RAG techniques that leverage new ingestion, retrieval, and generation patterns.  

RAG is a powerful solution used in tandem with the capabilities of LLMs. Check out our other examples for ideas on how you can build innovative solutions using RAGStack!